# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [49]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [50]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zorritos,-3.3318,-81.0562,22.58,85,94,5.91,PE,1698546313
1,1,punta arenas,-54.4893,-71.2913,-3.92,73,84,0.70,CL,1698546313
2,2,gore,-45.6420,169.2854,14.96,51,71,0.86,NZ,1698546315
3,3,porto velho,-8.6404,-62.9889,25.75,72,24,1.29,BR,1698546315
4,4,murghob,39.0794,73.7300,-13.72,74,77,1.70,TJ,1698546316


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    frame_width = 1000,
    frame_height = 600,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    scale = 0.55

)

# Display the map
plot_1

#Had issues displaying plot, the hvplot.show function works to compensate in this regard.
#It does launch a server to display it in a browser unfortunately
hvplot.show(plot_1)



Launching server at http://localhost:55979


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_ideal = city_data_df.loc[(city_data_df['Max Temp'] > 21) & \
                                   (city_data_df['Max Temp'] < 27) & \
                                   (city_data_df['Wind Speed'] < 4.5) & \
                                   (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_ideal.dropna()

# Display sample data
city_data_ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,tual,26.5277,-101.0251,24.16,56,0,1.67,MX,1698546327
60,60,sao vicente,-30.1417,128.6869,25.13,27,0,1.97,AU,1698546340
65,65,stettler,-28.6463,149.2710,26.76,21,0,2.89,AU,1698546341
71,71,polyarnyy,-26.3598,149.3778,24.89,26,0,2.90,AU,1698546344
91,91,itoman,-31.7346,134.5393,22.88,23,0,4.29,AU,1698546353


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,tual,MX,26.5277,-101.0251,56,
60,sao vicente,AU,-30.1417,128.6869,27,
65,stettler,AU,-28.6463,149.2710,21,
71,polyarnyy,AU,-26.3598,149.3778,26,
91,itoman,AU,-31.7346,134.5393,23,
131,umluj,MX,20.5026,-87.4560,87,
143,boysun,AR,-28.4932,-62.7454,60,
288,chernyakhovsk,MG,-20.0123,44.2484,80,
321,stefan voda,MX,26.5573,-111.1526,61,
341,port shepstone,CN,33.4151,113.3451,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [58]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'limit': 1,
    'categories': 'accommodation.hotel'

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


# With the random sample given only 1 hotel was found. 
# After testing with different radii the code was found to be sufficient.
# The only problem is the random selection which cannot be controlled.

Starting hotel search
tual - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
stettler - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
itoman - nearest hotel: No hotel found
umluj - nearest hotel: No hotel found
boysun - nearest hotel: Hotel Añatuya
chernyakhovsk - nearest hotel: No hotel found
stefan voda - nearest hotel: No hotel found
port shepstone - nearest hotel: No hotel found
oranjemund - nearest hotel: No hotel found
odda - nearest hotel: No hotel found
tauranga - nearest hotel: No hotel found
bethanie - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,tual,MX,26.5277,-101.0251,56,No hotel found
60,sao vicente,AU,-30.1417,128.6869,27,No hotel found
65,stettler,AU,-28.6463,149.2710,21,No hotel found
71,polyarnyy,AU,-26.3598,149.3778,26,No hotel found
91,itoman,AU,-31.7346,134.5393,23,No hotel found
131,umluj,MX,20.5026,-87.4560,87,No hotel found
143,boysun,AR,-28.4932,-62.7454,60,Hotel Añatuya
288,chernyakhovsk,MG,-20.0123,44.2484,80,No hotel found
321,stefan voda,MX,26.5573,-111.1526,61,No hotel found
341,port shepstone,CN,33.4151,113.3451,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    hover_cols = ['Country', 'Hotel Name'],
    frame_width = 1000,
    frame_height = 600,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    scale = 0.55

)

# Display the map

plot_2


hvplot.show(plot_2)

Launching server at http://localhost:56701
